# Deploying A Sentence-Embedding Model

## Contents

1. [Setup](#Setup)
2. [Deploy](#Deploy)


# Setup

- Load environmental variables
- Create Sagemaker Session
- Define S3 Bucket, path and model name
- Provide IAM role

In [2]:
%load_ext dotenv
import os
import sagemaker
from sagemaker.predictor import RealTimePredictor, json_serializer, json_deserializer
from sagemaker.pytorch import PyTorchModel

%dotenv -v

bucket = os.getenv("S3_BUCKET")
model_location = os.getenv("MODEL_PATH")   # path to model data
entry = os.getenv("ENTRY_POINT")     # Python file to execute
source = os.getenv("SOURCE_DIR")       # Source for code
version = os.getenv("FRAMEWORK")           # Pytorch Version
role = sagemaker.get_execution_role()      # IAM role


ValueError: Must setup local AWS configuration with a region supported by SageMaker.

# Deploy

Provide custom hosting functions:
- input_fn: Deserializes input data
- model_fn: Loads pytorch model
- predict_fn: run inference with input data
- output_fn: Serializes results of predict_fn
- predictor_cls: Creates the predictor with the endpoint name

In [ ]:
class JSONPredictor(RealTimePredictor):
    """Serializes and deserializes json
    """
    def __init__(self, endpoint_name, sagemaker_session):
        super(JSONPredictor, self).__init__(endpoint_name, sagemaker_session, json_serializer, json_deserializer)

In [ ]:
model = PyTorchModel(model_data=model_location,
                     role=role,
                     framework_version=version,
                     entry_point=entry,
                     source_dir=source,
                     predictor_cls=JSONPredictor)

In [ ]:
predictor = model.deploy(initial_instance_count=1,
                         instance_type='ml.m5.xlarge',
                         endpoint_name="sentence_similarity")

Help on class PyTorchModel in module sagemaker.pytorch.model:

class PyTorchModel(sagemaker.model.FrameworkModel)
 |  An PyTorch SageMaker ``Model`` that can be deployed to a SageMaker
 |  ``Endpoint``.
 |  
 |  Method resolution order:
 |      PyTorchModel
 |      sagemaker.model.FrameworkModel
 |      sagemaker.model.Model
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, model_data, role, entry_point, image=None, py_version='py3', framework_version=None, predictor_cls=<class 'sagemaker.pytorch.model.PyTorchPredictor'>, model_server_workers=None, **kwargs)
 |      Initialize an PyTorchModel.
 |      
 |      Args:
 |          model_data (str): The S3 location of a SageMaker model data
 |              ``.tar.gz`` file.
 |          role (str): An AWS IAM role (either name or full ARN). The Amazon
 |              SageMaker training jobs and APIs that create Amazon SageMaker
 |              endpoints use this role to access training data and model
 |         